# Get Paper Info
These notebook will get the paper info for each speakers in the conference. The attributes are: Number of papers (by year), first paper year, number of citation (by year), country

In [18]:
%run setup.py

### Load the data

In [8]:
dblp_name[8314]

'Barton%20A.%20Smith'

In [19]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
df = pd.read_csv('factProceedings.csv', encoding='utf-8')
dblp_name = df['Full name'].str.replace(' ', '%20')
dblp_name = dblp_name.drop_duplicates().reset_index(drop=True)

# drop nan values from the list
dblp_name = dblp_name.dropna().reset_index(drop=True)

In [22]:
dblp_name[:8176]

0              Isao%20%20Shirakawa
1              Tingting%20%20Hwang
2                 Sanjay%20%20Dhar
3               Dave%20J.%20Gurney
4                Michel%20%20Thill
                   ...            
8171            Fabio%20%20Pianesi
8172         Francesca%20%20Gennai
8173    Antonio%20%20Cristoforetti
8174             Elise%20van%20den
8175            Julia%20%20Schwarz
Name: Full name, Length: 8176, dtype: object

## dblp.org

In [23]:
df1 = pd.DataFrame(columns = ['Full name','Year','Year Count'])

for Author in tqdm(dblp_name[:8176]):
    author_list = []
    paper_list = []
    
    ##### Check if author is on dblp #####
    dblp_URL = "https://dblp.org/search?q=" + Author
    page = requests.get(dblp_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    control_check = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")
    if len(control_check) > 0:
        first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']
        # print(Author," is on the webpage and the process continues...")
        
        ##### Get info from author #####
        author_page = requests.get(first_author)
        soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

        Year_list = []

        sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
        for s in range(0,len(sections)): # Nu går vi ind på hver enkelt sektion
            rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc','entry incollection toc', 'entry book toc','entry editor toc','entry reference toc']})
            last_row_idx = 0
            
            for row in range(0,len(rows_in_sections)): # Nu går vi ind på hver enkelt række
                #### Append year multipliers
                if rows_in_sections[row].p == None:
                    Year_list.append(rows_in_sections[row].text)
                    paper_list.append(row-last_row_idx-1)
                    last_row_idx = row 
                if row == len(rows_in_sections)-1: # den sidste linje i hver sektion
                    paper_list.append(len(rows_in_sections)-last_row_idx-1)
            
            # remove all -1 values from the list
            paper_list = [x for x in paper_list if x >= 0]
                
        # create a dataframe
        df2 = pd.DataFrame(columns = ['Full name','Year','Year Count'])
        # append full name to the dataframe
        
        df2['Year'] = Year_list
        df2['Year Count'] = paper_list
        df2['Full name'] = Author
        df1 = df1.append(df2, ignore_index=True)
    time.sleep(1)
        

100%|██████████| 8176/8176 [9:18:30<00:00,  4.10s/it]     


In [28]:
df1

,Full name,Year,Year Count
0,Isao%20%20Shirakawa,2016,1
1,Isao%20%20Shirakawa,2014,2
2,Isao%20%20Shirakawa,2013,1
3,Isao%20%20Shirakawa,2011,1
4,Isao%20%20Shirakawa,2008,1
...,...,...,...
98089,Julia%20%20Schwarz,2014,5
98090,Julia%20%20Schwarz,2012,2
98091,Julia%20%20Schwarz,2011,3
98092,Julia%20%20Schwarz,2010,3


In [29]:
df1.to_csv("paper_count_per_year_proceedings_1.csv", index=False)

## Now we do stuff

In [30]:
df1 = pd.read_csv("paper_count_per_year_proceedings_2.csv", encoding='utf-8')
df2 = pd.read_csv("paper_count_per_year_proceedings_1.csv", encoding='utf-8')

df1 = df1.append(df2, ignore_index=True)

In [31]:
(df1['Full name'].nunique()/df['Full name'].nunique())*100

84.52145081072338

In [13]:
df2['Full name'].nunique()

373

In [32]:
# Find the names in df which is not i df1
df2 = dblp_name[~dblp_name.isin(df1['Full name'])]
df2

4608      Kenneth%20Mark%20Ross
9631        Asbjrn%20%20Følstad
9741          Sren%20%20Nielsen
10890        Sren%20S.%20Sander
13808       Alexa%20Frank%20Siu
                  ...          
96228    Valery%20A.%20Topinsky
96229         Damir%20%20Vandic
96230     Didier%20%20Nibbering
96439         Denis%20%20Gillet
96440       Danilo%20%20Menezes
Name: Full name, Length: 15312, dtype: object